In [3]:
%load_ext autoreload
%autoreload
from src.audio_generator import DataGenerator, get_wav_paths, get_labels
import numpy as np
import librosa
from kapre.composed import get_melspectrogram_layer
from tensorflow.io import read_file
from tensorflow.audio import decode_wav

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
wav, sr = librosa.load(file, sr=8000, res_type='kaiser_fast')
resamp = librosa.util.fix_length(wav, 80000)
resamp.shape

(80000,)

(46840,)

In [11]:
file = 'data/LibriSpeech/dev-clean/1272/128104/1272-128104-0000.wav'

In [5]:
wav, _ = decode_wav(file)

In [7]:
wav.numpy()

array([[0.00238037],
       [0.0020752 ],
       [0.00198364],
       ...,
       [0.00042725],
       [0.00057983],
       [0.0010376 ]], dtype=float32)

In [ ]:
get_melspectrogram_layer()

In [7]:
all_wavs, mask = get_wav_paths('data/LibriSpeech/dev-clean/',5)

In [8]:
len(all_wavs)

1118

In [9]:
labels, vocab = get_labels('data/LibriSpeech/dev-clean/', mask)

In [11]:
labels.shape

(1118, 4231)

In [14]:
labels

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
vocab

['abandoned',
 'abbess',
 'ability',
 'able',
 'abode',
 'about',
 'above',
 'abroad',
 'absolutely',
 'absorbed',
 'absurd',
 'abundant',
 'accept',
 'accepted',
 'accident',
 'accord',
 'accordingly',
 'accosted',
 'account',
 'accustomed',
 'aching',
 'acid',
 'acidity',
 'acquainted',
 'acquainting',
 'acquired',
 'across',
 'act',
 'action',
 'add',
 'added',
 'adding',
 'additional',
 'address',
 'addressed',
 'administrator',
 'admiration',
 'admission',
 'admit',
 'admitted',
 'admonishing',
 'adopted',
 'adrift',
 'advance',
 'advanced',
 'advancing',
 'advantage',
 'adventure',
 'adventures',
 'affairs',
 'affectionate',
 'affirm',
 'affirmation',
 'affirmed',
 'afford',
 'afraid',
 'afresh',
 'africa',
 'after',
 'afternoon',
 'afterwards',
 'again',
 'against',
 'age',
 'aged',
 'agglomerations',
 'aghast',
 'agile',
 'agility',
 'ago',
 'agree',
 'agreed',
 'agreeing',
 'agrees',
 'ah',
 'aid',
 'ain',
 'air',
 'akin',
 'alabama',
 'alarm',
 'alarmed',
 'alas',
 'alexander

In [6]:
librosa.get_duration(all_wavs[0])

ParameterError: Audio data must be floating-point

In [20]:
datagen = DataGenerator(all_wavs, labels, 8000, 5, len(vocab), batch_size=5, shuffle=True)

In [30]:
for item in datagen:
    for thing in item:
        for audio in thing:
            print(audio[0])

[-8.911266e-05]
[-0.00028142]
[-0.00440552]
[-6.066912e-05]
[-0.00027795]
0.0
0.0
1.0
0.0
0.0
[-0.00016533]
[-2.2396816e-05]
[-0.00070711]
[0.00144101]
[0.00043539]
0.0
0.0
0.0
0.0
0.0


In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization, SimpleRNN, Input, TimeDistributed, Dense, Embedding, LSTM, Conv1D, Flatten
from tensorflow.keras.models import Sequential, Model

In [9]:
input1 = Input(shape=([100000,1]))
x = Conv1D(32, 9, activation='relu')(input1)
x = Flatten()(x)
out = Dense(1, activation='softmax')(x)
model = Model(inputs=[input1], outputs=[out])
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 100000, 1)]       0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 99992, 32)         320       
_________________________________________________________________
flatten_2 (Flatten)          (None, 3199744)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 3199745   
Total params: 3,200,065
Trainable params: 3,200,065
Non-trainable params: 0
_________________________________________________________________


In [10]:
history = model.fit(datagen, epochs=5, verbose=1)

Epoch 1/5
19/84 [=====>........................] - ETA: 7:17 - loss: 0.9979 - accuracy: 0.0021

KeyboardInterrupt: 